In [1]:
# Import necessary libraries
from sklearn.linear_model import *
from sklearn.metrics import accuracy_score,log_loss, r2_score
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import *
from sklearn.compose import *
from sklearn.svm import SVC
from sklearn.tree import *
from sklearn.model_selection import *
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.discriminant_analysis import *
from sklearn.preprocessing import *
from sklearn.neighbors import *
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.linear_model import *
import os

warnings.simplefilter('ignore')

os.chdir('D:\Datasets')

conc = pd.read_csv('Concrete_Data.csv')
conc

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [2]:
x= conc.drop('Strength',axis=1)
y=conc['Strength']

In [3]:
# Split the data into training and testing sets with a test size of 30% and a random state of 24
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=24)

# Create a Decision Tree Regressor with a random state of 24
dtr = DecisionTreeRegressor(random_state=24)

# Create a Ridge Regression model
ridge = Ridge()

# Create a K-Nearest Neighbors Regressor
knn = KNeighborsRegressor()

# voting

In [4]:
# Create a Voting Regressor with the following models:
# - Decision Tree Regressor ('TREE')
# - Ridge Regression ('RIDGE')
# - K-Nearest Neighbors Regressor ('KNN')
# Assign weights to each model: 0.83, 0.57, 0.66 respectively
voting = VotingRegressor([('TREE', dtr), ('RIDGE', ridge), ('KNN', knn)],
                        weights=[0.83, 0.57, 0.66])

# Fit the Voting Regressor to the training data
voting.fit(x_train, y_train)

# Use the Voting Regressor to predict the target values for the test data
y_pred = voting.predict(x_test)

# Calculate the R-squared (coefficient of determination) score for the predictions
print("R-squared Score:", r2_score(y_test, y_pred))

R-squared Score: 0.8228648937143235


# calculating individual r2-score for all

In [5]:
# Create a Voting Regressor with only the Decision Tree Regressor model
voting = VotingRegressor([('TREE', dtr)])

# Fit the Voting Regressor to the training data
voting.fit(x_train, y_train)

# Use the Voting Regressor to predict the target values for the test data
y_pred = voting.predict(x_test)

# Calculate the R-squared (coefficient of determination) score for the predictions
print("R-squared Score:", r2_score(y_test, y_pred))

R-squared Score: 0.83089474226832


In [6]:
# Create a Voting Regressor with only the Ridge Regression model
voting = VotingRegressor([('RIDGE', ridge)])

# Fit the Voting Regressor to the training data
voting.fit(x_train, y_train)

# Use the Voting Regressor to predict the target values for the test data
y_pred = voting.predict(x_test)

# Calculate the R-squared (coefficient of determination) score for the predictions
print("R-squared Score:", r2_score(y_test, y_pred))

R-squared Score: 0.5771749099675625


In [7]:
# Create a Voting Regressor with only the K-Nearest Neighbors Regressor model
voting = VotingRegressor([('KNN', knn)])

# Fit the Voting Regressor to the training data
voting.fit(x_train, y_train)

# Use the Voting Regressor to predict the target values for the test data
y_pred = voting.predict(x_test)

# Calculate the R-squared (coefficient of determination) score for the predictions
print("R-squared Score:", r2_score(y_test, y_pred))

R-squared Score: 0.6693959113903629


# to check parameters

In [9]:
# Print the parameters of the Voting Regressor
voting.get_params()

{'estimators': [('KNN', KNeighborsRegressor())],
 'n_jobs': None,
 'verbose': False,
 'weights': None,
 'KNN': KNeighborsRegressor(),
 'KNN__algorithm': 'auto',
 'KNN__leaf_size': 30,
 'KNN__metric': 'minkowski',
 'KNN__metric_params': None,
 'KNN__n_jobs': None,
 'KNN__n_neighbors': 5,
 'KNN__p': 2,
 'KNN__weights': 'uniform'}

# grid search (hyperparameter tuning)

In [10]:
# Create a K-Fold cross-validation object with 5 splits, shuffling the data, and a random state of 24
kfold = KFold(n_splits=5, shuffle=True, random_state=24)

# Create a Voting Regressor with the following models:
# - Decision Tree Regressor ('TREE')
# - Ridge Regression ('RIDGE')
# - K-Nearest Neighbors Regressor ('KNN')
# Assign weights to each model: 0.83, 0.57, 0.66 respectively
voting = VotingRegressor([('TREE', dtr), ('RIDGE', ridge), ('KNN', knn)],
                        weights=[0.83, 0.57, 0.66])

# Define the parameter grid for hyperparameter tuning
params = {
    'TREE__max_depth': [None, 3],
    'TREE__min_samples_split': [2, 10],
    'TREE__min_samples_leaf': [1, 10],
    'RIDGE__alpha': [0.5, 1],
    'KNN__n_neighbors': [3, 5]
}

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Voting Regressor as the estimator, the parameter grid defined above, the K-Fold cross-validation,
# set the verbosity to 3
gcv = GridSearchCV(voting, param_grid=params, cv=kfold, verbose=3)

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score found by the GridSearchCV
print("Best Score:", gcv.best_score_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END KNN__n_neighbors=3, RIDGE__alpha=0.5, TREE__max_depth=None, TREE__min_samples_leaf=1, TREE__min_samples_split=2;, score=0.859 total time=   0.0s
[CV 2/5] END KNN__n_neighbors=3, RIDGE__alpha=0.5, TREE__max_depth=None, TREE__min_samples_leaf=1, TREE__min_samples_split=2;, score=0.844 total time=   0.0s
[CV 3/5] END KNN__n_neighbors=3, RIDGE__alpha=0.5, TREE__max_depth=None, TREE__min_samples_leaf=1, TREE__min_samples_split=2;, score=0.863 total time=   0.0s
[CV 4/5] END KNN__n_neighbors=3, RIDGE__alpha=0.5, TREE__max_depth=None, TREE__min_samples_leaf=1, TREE__min_samples_split=2;, score=0.788 total time=   0.0s
[CV 5/5] END KNN__n_neighbors=3, RIDGE__alpha=0.5, TREE__max_depth=None, TREE__min_samples_leaf=1, TREE__min_samples_split=2;, score=0.864 total time=   0.0s
[CV 1/5] END KNN__n_neighbors=3, RIDGE__alpha=0.5, TREE__max_depth=None, TREE__min_samples_leaf=1, TREE__min_samples_split=10;, score=0.844 total ti